In [1]:
import pandas as pd
import json
import datetime


In [5]:
file_path = './Class_Timetable.xlsx' #processing course data for timetable subpage
df = pd.read_excel(file_path)

In [9]:
df['START TIME'].dtypes

dtype('O')

In [10]:
first_row = df.iloc[0]
print(first_row)

TERM                                                 2025-26 Sem 1
ACAD_CAREER                                                   UGME
COURSE CODE                                               BBMS1001
CLASS SECTION                                                   1A
MON                                                            MON
TUE                                                            NaN
WED                                                            NaN
THU                                                            NaN
FRI                                                            NaN
SAT                                                            NaN
SUN                                                            NaN
VENUE                                                          LE3
START TIME                                                   14:00
END TIME                                                     15:50
COURSE TITLE          Introduction to Human Anatomy and Physio

In [14]:
courseData = []

for _, row in df.iterrows():
    if row['TERM'] is pd.NaT: continue 
    if row['TERM'][8:13] == 'Sem 2': continue #edit 'Sem 2' to 'Sem 1' to get course data for sem2
    courseName = row['COURSE CODE'] + '-' + str(row['CLASS SECTION'])
    courseTitle = row['COURSE TITLE']
    
    days = row.loc[['MON', 'TUE', 'WED', 'THU', 'FRI', 'SAT', 'SUN']].tolist()
    day = [x for x in days if pd.notna(x)]
    if len(day) == 0 : continue
    else: day = day[0]
        
    venue = row['VENUE']
    if type(venue) == float : venue = 'TBA'
    
    if row['START TIME'] is pd.NaT: start_time = ''
    if row['END TIME'] is pd.NaT: end_time = ''
    # if isinstance(row['START TIME'], datetime.time) : start_time = row['START TIME'].strftime("%H:%M")
    # if isinstance(row['END TIME'], datetime.time) :end_time = row['END TIME'].strftime("%H:%M")

    start_time = row['START TIME']
    end_time = row['END TIME']
    time = f"{start_time}-{end_time}"
    
    course = next((c for c in courseData if c['courseName'] == courseName), None)
    
    if course is None:
        course = {
            'courseName': courseName,
            'courseTitle': courseTitle,
            'lectures': [],
            'isChecked': False,
        }
        courseData.append(course)
        
    lecture = {
        'day':day,
        'time': time,
        'venue': venue
    }
    course['lectures'].append(lecture)
    
for course in courseData:
    course['lectures'] = [dict(t) for t in {tuple(d.items()) for d in course['lectures']}]
    
json_string = json.dumps(courseData)

print(json_string)

[{"courseName": "BBMS1001-1A", "courseTitle": "Introduction to Human Anatomy and Physiology", "lectures": [{"day": "MON", "time": "14:00-15:50", "venue": "TBA"}, {"day": "THU", "time": "16:00-17:50", "venue": "LE3"}, {"day": "MON", "time": "14:00-15:50", "venue": "FMLMDL1"}, {"day": "THU", "time": "16:00-17:50", "venue": "FMLMDL1"}, {"day": "MON", "time": "14:00-15:50", "venue": "LE3"}], "isChecked": false}, {"courseName": "BBMS2004-1A", "courseTitle": "Human Anatomy", "lectures": [{"day": "TUE", "time": "10:00-11:50", "venue": "TBA"}, {"day": "THU", "time": "10:00-11:50", "venue": "LE2"}, {"day": "THU", "time": "10:00-11:50", "venue": "TBA"}, {"day": "TUE", "time": "10:00-11:50", "venue": "FMLMDL2"}, {"day": "TUE", "time": "10:00-11:50", "venue": "LE2"}, {"day": "THU", "time": "10:00-11:50", "venue": "FMLMDL1"}], "isChecked": false}, {"courseName": "BBMS2006-1A", "courseTitle": "Physiological Basis of Health and Disease", "lectures": [{"day": "FRI", "time": "09:00-10:50", "venue": "MB

In [15]:
filename = 'courseDataSem1.json'  # Specify the filename or path as desired
with open(filename, 'w') as file:
    file.write(json_string)

print(f"JSON data saved to {filename}")

JSON data saved to courseDataSem1.json
